# Text Data Normalization & Visualization using Tableau

We need create a web dashboard prototype in Python that allows users to:

- Create slicers and dicers
- Filters by date range / ...
- Show data both in table and plotted formats
- Use paid_amount column for analysis and other columns for filters

For that, the normalization of text values in columns is needed. We have to think about update this dashboard monthly and do not have time to normalize values manually.

In [1]:
import re

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer 
from scipy.spatial import distance

In [2]:
def norm_service_cat(row):
    row1 = row.replace('Specialists', 'Specialist')
    for word in service_cat_words:
        row2 = row1.replace(word, (word+" "))
        row1 = row2
    return row1.strip()

In [3]:
def clear_speciality(row):
    row1 = row.replace("\t","").replace("&"," AND ").replace("/", " OR ").replace(",", " ")
    row2 = re.split(' - ', row1)
    row3 = row2[-1]
    row4 = re.split('[^a-z,]', row3)
    row5 = []
    row5.append([x for x in row4 if x])
    row6 = " ".join(row5[0])
    return row6

In [4]:
def replace_sinonim(row):
    row1 = re.split(' ', row)
    for x in row1:
        if x in second:
            row1[row1.index(x)] = first[second.index(x)]
        row2 = ' '.join(row1)
    return row2

In [5]:
def transform_speciality(row):
    if row in Second:        
        return First[Second.index(row)]
    else:
        return row

## 1. Data load

In [6]:
data = pd.read_csv('claims_data.csv')
data.head()

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT
0,201801,AncillaryFFS,NaN,Payer F,128438
1,201801,AncillaryFFS,NaN,Payer H,66624
2,201801,AncillaryFFS,NaN,Payer O,118108
3,201801,AncillaryFFS,NaN,Payer W,1073
4,201801,AncillaryFFS,ACH,Payer W,604


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52152 entries, 0 to 52151
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   MONTH             52152 non-null  int64 
 1   SERVICE_CATEGORY  52152 non-null  object
 2   CLAIM_SPECIALTY   51901 non-null  object
 3   PAYER             52152 non-null  object
 4   PAID_AMOUNT       52152 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


## 2. Year & Month normalize

For further dashboard creating we need to take out the month and year in separate columns *MONTH* and *YEAR*

In [8]:
data['MONTH'] = data['MONTH'].astype(str)
data['YEAR'] = data['MONTH'].apply(lambda x : x[:4])
data['MONTH'] = data['MONTH'].apply(lambda x : x[4:])

In [9]:
data['MONTH'].value_counts()

01    5284
03    5261
05    5250
02    5126
04    5020
06    4835
07    3650
10    3611
08    3603
09    3541
12    3490
11    3470
00      11
Name: MONTH, dtype: int64

In [10]:
# replace unvalid month with 'NA'
data['MONTH'] = data['MONTH'].str.replace('00', 'NA', regex=True)

In [11]:
data['YEAR'].value_counts()

2019    21494
2018    20778
2020     9880
Name: YEAR, dtype: int64

In [12]:
data.head()

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT,YEAR
0,01,AncillaryFFS,NaN,Payer F,128438,2018
1,01,AncillaryFFS,NaN,Payer H,66624,2018
2,01,AncillaryFFS,NaN,Payer O,118108,2018
3,01,AncillaryFFS,NaN,Payer W,1073,2018
4,01,AncillaryFFS,ACH,Payer W,604,2018


## 2. SERVICE_CATEGORY normalization

In [13]:
data['SERVICE_CATEGORY'].value_counts()

SpecialistsFFS        12296
AncillaryFFS           9701
InpatientServices      9424
OutpatientServices     7582
ERServices             3696
ASCServices            2643
SNFServices            2501
SpecialistFFS          1684
PCPFFS                 1403
PCPEncounter           1222
Name: SERVICE_CATEGORY, dtype: int64

Let's decompose the phrases into simple words, create a list of these words and use it to normalize *SERVICE_CATEGORY*.

In [14]:
service_cat_words = ['Ancillary', 
                     'ASC', 
                     'Encounter', 
                     'ER', 
                     'FFS', 
                     'Inpatient', 
                     'Outpatient', 
                     'Services', 
                     'SNF', 
                     'Specialist', 
                     'PCP']

In [15]:
data['SERVICE_CATEGORY'] = data['SERVICE_CATEGORY'].apply(norm_service_cat)

In [16]:
data.head()

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT,YEAR
0,01,Ancillary FFS,NaN,Payer F,128438,2018
1,01,Ancillary FFS,NaN,Payer H,66624,2018
2,01,Ancillary FFS,NaN,Payer O,118108,2018
3,01,Ancillary FFS,NaN,Payer W,1073,2018
4,01,Ancillary FFS,ACH,Payer W,604,2018


## 3. CLAIM_SPECIALTY normalization

In [17]:
data['CLAIM_SPECIALTY'].value_counts()

INTERNAL MEDICINE                                     1227
FAMILY PRACTICE                                       1128
GENERAL PRACTICE                                       949
GENERAL SURGERY                                        806
PATHOLOGY                                              772
                                                      ... 
Pathology - Clinical Pathology/Laboratory Medicine       1
PREVENTIVE MEDICINE - ADDICTION MEDICINE                 1
ORTHOTICS/PROS FITTE                                     1
Nurse Practitioner - Obstetrics & Gynecology\t           1
Neonatology                                              1
Name: CLAIM_SPECIALTY, Length: 905, dtype: int64

At first sight, we have 905 different categories. First, let's try to clean up a little our data: we will bring it to a single register, remove unnecessary characters, also in the lines where the subcategory is indicated, for example: *Internal Medicine - Cardiovascular Disease*, we will leave only the last component: *Cardiovascular Disease.*

In [18]:
data['CLAIM_SPECIALTY'].fillna('unknown',inplace=True)
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].str.lower()

In [19]:
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].apply(clear_speciality)
data['CLAIM_SPECIALTY'].value_counts()

internal medicine               1730
family practice                 1512
radiology                       1180
general practice                1132
gastroenterology                1115
                                ... 
peder                              1
neurdi                             1
pediatric nurse practitioner       1
psynur                             1
registered                         1
Name: CLAIM_SPECIALTY, Length: 740, dtype: int64

Now there are 740 categories. A closer look shows that many words are very similar, some words are simply cut off. Let's try to identify such "synonyms". Let's make a list of all words and words with a length of at least 6 characters will be considered synonyms if the first 5 letters in the word match.

In [20]:
# list of all encountered specializations

spec_list1 = list(data['CLAIM_SPECIALTY'].value_counts().index)
print('Total specialities: ', len(spec_list1))
spec_list1[:10]

Total specialities:  740


['internal medicine',
 'family practice',
 'radiology',
 'general practice',
 'gastroenterology',
 'pathology',
 'general surgery',
 'cardiology',
 'anesthesiology',
 'orthopedic surgery']

In [21]:
# list of all encountered words of specializations

vectorizer1 = CountVectorizer()     # unigrams
X1 = vectorizer1.fit_transform(spec_list1)
word_list1 = vectorizer1.get_feature_names()
print('Total words in specialities: ', len(word_list1))
word_list1[:10]

Total words in specialities:  577


['abulatory',
 'abuse',
 'access',
 'accup',
 'ach',
 'acupu',
 'acupuncturist',
 'acute',
 'addiction',
 'addmed']

In [22]:
# select words with a length of at least 6 letters for reliability

words6 = []
for word in word_list1:
    if len(word)>6:
        words6.append(word)
print('Total words with lenght > 6: ',len(words6))

Total words with lenght > 6:  263


In [23]:
# Find words that match the first 5 letters and write out their pairs

first = []
second = []
for i in range(len(words6)):
    for j in range(i+1, len(words6)):
        if (words6[i][:5] == words6[j][:5]) and (words6[j] not in second):
            first.append(words6[i])
            second.append(words6[j])
print(len(first))

92


Let's see how it works on the example of the first 10 "synonyms":

In [24]:
for a, b in zip(first[:10], second[:10]):
    print(a, '     ', b)

ambulance       ambulatory
anesthesia       anesthesiologist
anesthesia       anesthesiology
anesthesia       anesthetist
audiologist       audiology
audiologist       audiometry
behavior       behavioral
cardiac       cardiatric
cardiac       cardiologist
cardiac       cardiology


Replace words from the second list with words from the first list.

In [25]:
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].apply(replace_sinonim)
data['CLAIM_SPECIALTY'].value_counts()

internal medical                  1730
family practice                   1512
neurological                      1449
hospice                           1440
radiology                         1180
                                  ... 
nponc                                1
hep                                  1
blood bank transfu                   1
neonatology                          1
registered nurse private nurse       1
Name: CLAIM_SPECIALTY, Length: 696, dtype: int64

There are less categories, but not much less (696 instead of 740). Some of them just differ in the words order among themselves. Let's try to find them using the cosine measure.

In [26]:
spec_list2 = list(data['CLAIM_SPECIALTY'].value_counts().index)
print('Total specialities now: ', len(spec_list2))

Total specialities now:  696


In [27]:
vectorizer2 = CountVectorizer()     # unigrams
X2 = vectorizer2.fit_transform(spec_list2)
words2 = vectorizer2.get_feature_names()

vectors2 = X2.toarray()
vectors2.shape

(696, 485)

Let's detach out categories with cosine measure <0.3 that means the categories are similar

In [28]:
First = []
Second = []
Dist = []
for i in range(len(spec_list2)):
    for j in range(i+1, len(spec_list2)):
        cosi = distance.cosine(vectors2[i], vectors2[j])
        if (cosi < 0.3):
            First.append(spec_list2[i])
            Second.append(spec_list2[j])
            Dist.append(cosi)
print('Total similar categories: ', len(Dist))

Total similar categories:  472


Let's see how it works on the example:

In [29]:
for a, b, c in zip(First[:10], Second[:10], Dist[:10]):
    print(a, '     ', b, '     ', c)

internal medical       physical internal medical       0.18350341907227385
internal medical       endocrinology internal medical       0.18350341907227385
internal medical       rheumatology internal medical       0.18350341907227385
internal medical       oncologic internal medical       0.18350341907227385
internal medical       internal pain medical       0.18350341907227385
internal medical       internal medical geriatric       0.18350341907227385
internal medical       medical       0.29289321881345254
family practice       family nurse practice       0.18350341907227385
family practice       family       0.29289321881345254
family practice       family practice nurse prac       0.29289321881345254


Let's replace the categories from the *Second* list with the categories from the *First* list

In [30]:
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].apply(transform_speciality)
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].str.upper()
data['CLAIM_SPECIALTY'].value_counts()

CARDIAC                     2912
RADIOLOGY                   2313
INTERNAL MEDICAL            1889
FAMILY PRACTICE             1764
NEUROLOGICAL                1736
                            ... 
LTCH                           1
TBICRS                         1
GENET                          1
LARYNGOLOGY                    1
BEHAVIOR HEALTH FACILITY       1
Name: CLAIM_SPECIALTY, Length: 448, dtype: int64

Now there are 448 categories yet. Let's replace rare categories with 'OTHER SPECIALITY'

In [31]:
# Most popular CLAIM_SPECIALTY > S

S = 10  # frequency of popularity
spec_list3 = list((data['CLAIM_SPECIALTY'].value_counts()[(data['CLAIM_SPECIALTY'].value_counts())>S]).index)
print('Total categories that occur more than 10 times: ', len(spec_list3))
spec_list3[:10]

Total categories that occur more than 10 times:  246


['CARDIAC',
 'RADIOLOGY',
 'INTERNAL MEDICAL',
 'FAMILY PRACTICE',
 'NEUROLOGICAL',
 'HEMATOLOGY ONCOLOGIC',
 'HOSPICE',
 'ANESTHESIA',
 'GENERAL PRACTICE',
 'PATHOLOG']

In [32]:
data['CLAIM_SPECIALTY'] = data['CLAIM_SPECIALTY'].apply(lambda x: x if x in spec_list3 else  'OTHER SPECIALITY' )
data['CLAIM_SPECIALTY'].value_counts()

CARDIAC                                   2912
RADIOLOGY                                 2313
INTERNAL MEDICAL                          1889
FAMILY PRACTICE                           1764
NEUROLOGICAL                              1736
                                          ... 
GERPSY                                      11
GERIATRIC MEDICAL                           11
UROONC                                      11
OCCUPATIONAL MEDICAL                        11
END STAGE RENAL DISEASE ESRD TREATMENT      11
Name: CLAIM_SPECIALTY, Length: 247, dtype: int64

The total number of categories has become 247. You can change the frequency threshold depending on how much you can cut this parameter for further visualization.

## 4. PAYER normalization

In [33]:
data['PAYER'].value_counts()

Payer F     9020
Payer CA    8681
Payer W     7105
Payer UN    6526
Payer O     6278
Payer S     4189
Payer CO    3677
Payer B     3631
Payer H     2360
Payer UL     685
Name: PAYER, dtype: int64

Just remove *Payer*

In [34]:
data['PAYER'] = data['PAYER'].str.replace('Payer ', '', regex=True)

In [35]:
data.head()

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT,YEAR
0,01,Ancillary FFS,UNKNOWN,F,128438,2018
1,01,Ancillary FFS,UNKNOWN,H,66624,2018
2,01,Ancillary FFS,UNKNOWN,O,118108,2018
3,01,Ancillary FFS,UNKNOWN,W,1073,2018
4,01,Ancillary FFS,OTHER SPECIALITY,W,604,2018


Csv exported for further visualization.

## 5. Data export

In [36]:
data.to_csv('claims_export.csv')

## 6. Data Visualization

After processing, the data was visualized using Tableau tool [Link](https://public.tableau.com/views/Test_Task2/Dashboard3?:language=en-GB&:display_count=y&publish=yes&:origin=viz_share_link)